### Урок 2

#### Парсинг HTML. BeautifulSoup, MongoDB

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

*Наименование вакансии
*Предлагаемую зарплату (отдельно мин. и и отдельно макс.)
*Ссылку на саму вакансию        
*Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import json
import pandas as pd
from pprint import pprint

# https://russia.superjob.ru/vacancy/search/?keywords=Python

def _parser_superjob(vacancy):
    vacancy_data = []
    main_link = 'https://russia.superjob.ru'
    link = 'https://www.superjob.ru/vacancy/search/'
    params = {'keywords': vacancy,
              'profession_only': '1',
              'geo[c][0]': '15',
              'geo[c][1]': '1',
              'geo[c][2]': '9',
              'page': '',
              'noGeo': '1'
              }
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

    response = requests.get(main_link + '/vacancy/search/', params=params, headers=headers)
    if response.ok:
       soup = bs(response.text, 'html.parser')
       page_block = soup.find('a', {'class': 'f-test-button-1'})
    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())

    for page in range(0, last_page + 1):
        params['page'] = page
        response = requests.get(link, params=params, headers=headers)

        if response.ok:
            parsed_response = bs(response.text, 'html.parser')
            vacancy_items = parsed_response.find_all('div', {'class': 'f-test-vacancy-item'})

            for item in vacancy_items:
                vacancy_data.append(_parser_item_superjob(item))

    return vacancy_data

def _parser_item_superjob(item):

        vacancy_data = {}

        # vacancy_name
        vacancy_name = item.find_all('a')
        vacancy_data['vacancy_name'] = vacancy_name[0].getText()

        # salary
        salary = item.find('span', {'class': 'f-test-text-company-item-salary'}).findChild().text
        if salary == 'По договорённости':
            salary_min = None
            salary_max = None
            salary_currency = None
        else:
            salary = salary.replace(u'\xa0', ' ').split()
            salary_currency = salary.pop()
            if salary[0] == 'от':
                salary.remove('от')
                salary_min = float(''.join(salary[i] for i in range(len(salary))))
                salary_max = None
            elif salary[0] == 'до':
                salary.remove('до')
                salary_min = None
                salary_max = float(''.join(salary[i] for i in range(len(salary))))
            else:
                salary = ''.join(salary[i] for i in range(len(salary))).split('—')
                salary_min = float(salary[0])
                salary_max = float(salary[1]) if len(salary) > 1 else float(
                    salary[0])
        vacancy_data['salary_min'] = salary_min
        vacancy_data['salary_max'] = salary_max
        vacancy_data['salary_currency'] = salary_currency

        # link
        vacancy_link = item.find_all('a')
        vacancy_link = vacancy_link[0]['href']
        vacancy_data['vacancy_link'] = f'https://www.superjob.ru{vacancy_link}'

        # site
        vacancy_data['site'] = 'www.superjob.ru'

        return vacancy_data


def parser_vacancy(vacancy):
    vacancy_data = []
    vacancy_data.extend(_parser_superjob(vacancy))

    df = pd.DataFrame(vacancy_data)
    return df

vacancy = 'Python'
df = parser_vacancy(vacancy)

df[1:100]

,vacancy_name,salary_min,salary_max,salary_currency,vacancy_link,site
1,Вакансия: Backend-разработчик (Middle Python),NaN,NaN,None,https://www.superjob.ru/vakansii/vakansiya-360...,www.superjob.ru
2,Backend-разработчик (Middle Python Scrapy),NaN,NaN,None,https://www.superjob.ru/vakansii/backend-razra...,www.superjob.ru
3,Python senior developer,200000.0,NaN,руб.,https://www.superjob.ru/vakansii/python-senior...,www.superjob.ru
4,Python Team lead,300000.0,NaN,руб.,https://www.superjob.ru/vakansii/python-team-l...,www.superjob.ru
5,Junior Python разработчик,50000.0,NaN,руб.,https://www.superjob.ru/vakansii/junior-python...,www.superjob.ru
6,Middle Python разработчик,60000.0,NaN,руб.,https://www.superjob.ru/vakansii/middle-python...,www.superjob.ru
7,Инженер-программист Python (data science),NaN,NaN,None,https://www.superjob.ru/vakansii/inzhener-prog...,www.superjob.ru
8,Python developer middle,170000.0,NaN,руб.,https://www.superjob.ru/vakansii/python-develo...,www.superjob.ru
9,Программист-разработчик Python,NaN,NaN,None,https://www.superjob.ru/vakansii/programmist-r...,www.superjob.ru
10,BackEnd Python-разработчик,150000.0,180000.0,руб.,https://www.superjob.ru/vakansii/backend-pytho...,www.superjob.ru
